In [1]:
import json
import os
import re
import pandas as pd
from emoatlas import EmoScores
import networkx as nx
import numpy as np
from pprint import pprint
from scipy.spatial.distance import cosine
from collections import defaultdict

In [2]:
base_dir = "localdb"
valence_dir = "Valence_Metrics"
language='ita'

emos = EmoScores()
emosita = EmoScores(language='italian')

# Folders of interest
folders_of_interest = ['math']


In [3]:
models_data={}

for folder in folders_of_interest:
    folder_path = os.path.join(base_dir, folder)

    for filename in os.listdir(folder_path):
        
        fmnts=[]
        texts=[]

        file_path = os.path.join(folder_path, filename)
        
        if language=='ita':
            if '(ITA)' in file_path:
                with open(file_path, 'r') as file:
                    for line in file:
                        json_obj = json.loads(line)
                        fmnts.append(json_obj['fmnt']['syntactic'])
                        texts.append(' '.join(json_obj['lemmatized_test']))
                models_data[filename.rstrip('.jsonl')] = { 'Network': emosita.combine_formamentis(fmnts, weights=True),
                                         'Texts': ' '.join(texts)
                }              
        if language=='eng':
            if '(ITA)' not in file_path:
                with open(file_path, 'r') as file:
                    for line in file:
                        json_obj = json.loads(line)
                        fmnts.append(json_obj['fmnt']['syntactic'])
                        texts.append(' '.join(json_obj['lemmatized_test']))
                models_data[filename.rstrip('.jsonl')] = { 'Network': emos.combine_formamentis(fmnts, weights=True),
                                         'Texts': ' '.join(texts)
                }              


In [4]:
def create_graph(network_data):
    G = nx.Graph()
    for edge in network_data:
        G.add_edge(edge[0], edge[1], weight=edge[2])
    return G

def compute_metrics(G, text):
    strength = dict(nx.degree(G, weight='weight'))
    degree = dict(nx.degree(G))
    closeness = nx.closeness_centrality(G)
    
    # Compute word frequencies
    word_frequencies = defaultdict(int)
    words = text.split()  # Split text into words
    
    for word in words:
        word_frequencies[word] += 1  # Count frequency of each word 
    word_frequencies=dict(word_frequencies)
    return strength, degree, closeness, word_frequencies



In [5]:
def analyze_models():
    results = {model:{} for model in models_data}
    all_tf_words = {}
    all_degree_words={}

    for model in models_data:
        print('doing ', model)
        G = create_graph(models_data[model]['Network'])
        strength, degree, closeness, tf = compute_metrics(G, models_data[model]['Texts'])
        
        results[model]['strength'] = strength
        results[model]['degree'] = degree
        results[model]['closeness'] = closeness
        results[model]['tf'] = tf

        all_tf_words[model] = set(degree.keys())

    common_words = set.intersection(*all_tf_words.values())
    
    # Filter out words not used in all models
    for model in results:
        results[model]['strength'] = {word: value for word, value in results[model]['strength'].items() if word in common_words}
        results[model]['degree'] = {word: value for word, value in results[model]['degree'].items() if word in common_words}
        results[model]['closeness'] = {word: value for word, value in results[model]['closeness'].items() if word in common_words}
        results[model]['tf'] = {word: value for word, value in results[model]['tf'].items() if word in common_words}

    all_words_used = set().union(*all_degree_words.values(), *all_tf_words.values())
    words_not_in_all_models = all_words_used - common_words
    if words_not_in_all_models:
        print("Words not used by every model:", words_not_in_all_models)
    
    return results

results = analyze_models()



doing  GPT-3.5
doing  GPT-4
doing  Haiku
doing  Llama-3-8B
doing  Llama-3.1-70B
doing  Mistral-7b
Words not used by every model: {'recommend', 'recipient', 'middle', 'unwelcome', 'inquire', 'knowledgeable', 'impossible', 'discrepancy', 'perpetuating', 'embody', 'importally', 'not-identify', 'indecipherable', 'learnable', 'customize', 'hit', 'exclusively', 'continued', 'phenomena', 'conversation', 'dearth', 'rome', 'continuous', 'figment', 'disorganization', 'attuned', 'identity', 'lerner', 'evasion', 'pedagogically', 'mathematicians', 'untrue', 'spark', 'inst]<<sys>>[/inst', 'practitioner', 'agent', 'political', 'means', 'spot', 'statistical', 'urge', 'classmate', 'miller', 'havoc', 'character', 'mysterious', 'not-math', 'unlearnable', 'enrol', 'sphere', 'ladd', 'opening', 'preventable', 'alternatively', 'hodgins', 'distinguish', 'unvarnished', 'weigh', 'ceiling', 'downplay', 'fortunately', 'occur', 'double', 'schoenfeld', 'falsely', 'prioritization', 'dash', 'surface', 'supporter', 'p

In [6]:
def clean_dictionary_keys(data):
    cleaned_data = {}
    
    # Find common words across all models and all inner dictionaries
    all_word_sets = [
        set(inner_dict.keys())
        for model_data in data.values()
        for inner_dict in model_data.values()
    ]
    common_words = set.intersection(*all_word_sets)
    
    print(f"Number of common words across all models: {len(common_words)}")
    
    for model, model_data in data.items():
        print(f"Model: {model}")
        
        # Create a new dictionary for this model with only common words
        cleaned_data[model] = {
            inner_dict_type: {word: value for word, value in inner_dict.items() if word in common_words}
            for inner_dict_type, inner_dict in model_data.items()
        }
        
        # Print the length of each metric after cleaning
        for metric_name, metric_values in cleaned_data[model].items():
            print(f"  Metric: {metric_name}, Len: {len(metric_values)}")
        
        print("---")
    
    return cleaned_data

results=clean_dictionary_keys(results)
results

Number of common words across all models: 1030
Model: GPT-3.5
  Metric: strength, Len: 1030
  Metric: degree, Len: 1030
  Metric: closeness, Len: 1030
  Metric: tf, Len: 1030
---
Model: GPT-4
  Metric: strength, Len: 1030
  Metric: degree, Len: 1030
  Metric: closeness, Len: 1030
  Metric: tf, Len: 1030
---
Model: Haiku
  Metric: strength, Len: 1030
  Metric: degree, Len: 1030
  Metric: closeness, Len: 1030
  Metric: tf, Len: 1030
---
Model: Llama-3-8B
  Metric: strength, Len: 1030
  Metric: degree, Len: 1030
  Metric: closeness, Len: 1030
  Metric: tf, Len: 1030
---
Model: Llama-3.1-70B
  Metric: strength, Len: 1030
  Metric: degree, Len: 1030
  Metric: closeness, Len: 1030
  Metric: tf, Len: 1030
---
Model: Mistral-7b
  Metric: strength, Len: 1030
  Metric: degree, Len: 1030
  Metric: closeness, Len: 1030
  Metric: tf, Len: 1030
---


{'GPT-3.5': {'strength': {'anxiety': 29251,
   'issue': 10625,
   'pervasive': 323,
   'hinder': 4931,
   'success': 2835,
   'student': 36386,
   'subject': 4684,
   'enjoyment': 39,
   'belief': 1980,
   'math': 29606,
   'real': 5867,
   'concern': 859,
   'impact': 7895,
   'significant': 4069,
   'performance': 5522,
   'academic': 3397,
   'overall': 4336,
   'experience': 9694,
   'feel': 6231,
   'they': 4864,
   'overwhelmed': 647,
   'stress': 2425,
   'task': 4523,
   'fearful': 205,
   'mathematical': 3585,
   'face': 5044,
   'lead': 6578,
   'motivation': 1152,
   'behavior': 261,
   'decrease': 1089,
   'avoidance': 3207,
   'attitude': 8040,
   'pressure': 3588,
   'stem': 1424,
   'combination': 354,
   'factor': 1846,
   'past': 2650,
   'negative': 5635,
   'lack': 3827,
   'societal': 2060,
   'perception': 999,
   'confidence': 11412,
   'excel': 1229,
   'difficult': 1306,
   'abstract': 151,
   'create': 15345,
   'cycle': 2264,
   'fulfil': 194,
   'prophecy': 2

In [7]:
wordstoremove={'math':['math','anxiety','matematica','ansia'],
               'climate':['climate','change','cambiamento','climatico'],
               'gwarming':['global','warming','riscaldamento','globale'],
               'misinformation_health':['misinformation','health','disinformazione','salute','bufala','teoria','complotto','information','theory','conspiracy'],}

def remove_keys(d):
    if isinstance(d, dict):
        # Filter out keys named 'climate' or 'change'
        d = {k: remove_keys(v) for k, v in d.items() if k.lower() not in wordstoremove[folders_of_interest[0]]}
    return d

results2=remove_keys(results)

In [8]:
def compute_euclidean_differences(results):

    import math
    # Calculate Euclidean distance for 'strength', 'degree', 'closeness', and 'tf' for each model
    for model in results:
        other_models = [m for m in results if m != model]
        
        for node in results[model]['strength']:
            distances = [(results[model]['strength'][node] - results[other_model]['strength'].get(node, 0))**2 for other_model in other_models]
            euclidean_distance = math.sqrt(sum(distances))
            results[model].setdefault('diff_strength', {})[node] = euclidean_distance
        
        for node in results[model]['degree']:
            distances = [(results[model]['degree'][node] - results[other_model]['degree'].get(node, 0))**2 for other_model in other_models]
            euclidean_distance = math.sqrt(sum(distances))
            results[model].setdefault('diff_degree', {})[node] = euclidean_distance

        for node in results[model]['closeness']:
            distances = [(results[model]['closeness'][node] - results[other_model]['closeness'].get(node, 0))**2 for other_model in other_models]
            euclidean_distance = math.sqrt(sum(distances))
            results[model].setdefault('diff_closeness', {})[node] = euclidean_distance
        
        for word in results[model]['tf']:
            distances = [(results[model]['tf'][word] - results[other_model]['tf'].get(word, 0))**2 for other_model in other_models]
            euclidean_distance = math.sqrt(sum(distances))
            results[model].setdefault('diff_tf', {})[word] = euclidean_distance

    # Example output for checking
    for model in results:
        print(f"Model: {model}")
        if 'believe' in results[model]['diff_strength']:
            print(f"Euclidean Distance Strength for 'believe': {results[model]['diff_strength']['believe']}")
        if 'believe' in results[model]['diff_degree']:
            print(f"Euclidean Distance Degree for 'believe': {results[model]['diff_degree']['believe']}")
        if 'believe' in results[model]['diff_closeness']:
            print(f"Euclidean Distance Closeness for 'believe': {results[model]['diff_closeness']['believe']}")
        if 'believe' in results[model]['diff_tf']:
            print(f"Euclidean Distance TF for 'believe': {results[model]['diff_tf']['believe']}")
    return results

In [9]:
euclidean_differences=compute_euclidean_differences(results2)

Model: GPT-3.5
Euclidean Distance Strength for 'believe': 7548.177594625076
Euclidean Distance Degree for 'believe': 352.6159383805559
Euclidean Distance Closeness for 'believe': 0.0516479765248441
Euclidean Distance TF for 'believe': 1535.5021979795406
Model: GPT-4
Euclidean Distance Strength for 'believe': 14875.229914189562
Euclidean Distance Degree for 'believe': 254.8411269791436
Euclidean Distance Closeness for 'believe': 0.05113824586402233
Euclidean Distance TF for 'believe': 2904.4564035288945
Model: Haiku
Euclidean Distance Strength for 'believe': 7327.529802054714
Euclidean Distance Degree for 'believe': 245.63794495150785
Euclidean Distance Closeness for 'believe': 0.0541425760534684
Euclidean Distance TF for 'believe': 1532.587028523992
Model: Llama-3-8B
Euclidean Distance Strength for 'believe': 9337.705981663805
Euclidean Distance Degree for 'believe': 339.6733725213091
Euclidean Distance Closeness for 'believe': 0.04017706604466737
Euclidean Distance TF for 'believe': 1

In [10]:
euclidean_differences.keys()

dict_keys(['GPT-3.5', 'GPT-4', 'Haiku', 'Llama-3-8B', 'Llama-3.1-70B', 'Mistral-7b'])

In [11]:
def create_standardized_dataframe(results):
    # Step 1-4: Create initial DataFrame and calculate z-scores (unchanged)
    df = pd.concat({k: pd.DataFrame(v['strength'].items(), columns=['node', 'strength']) for k, v in results.items()}, axis=1)

    display(df[df.isna().any(axis=1)])

    # Delete rows with NaN values
    df = df.dropna()

    for model in results:
        df[(model, 'closeness')] = df[(model, 'node')].map(results[model]['closeness'])
        df[(model, 'degree')] = df[(model, 'node')].map(results[model]['degree'])
        df[(model, 'tf')] = df[(model, 'node')].map(results[model]['tf'])
    
    for model in results:
        #df[(model, 's')] = (df[(model, 'strength')]/1000).round(1)
        df[(model, 'd')] = (df[(model, 'degree')]/100).round(1)
        df[(model, 'f')] = (df[(model, 'tf')]/1000).round(1)
        df[(model, 'c')] = df[(model, 'closeness')].round(2)

    for model in results:
        #df[(model, 'Ds')] = df[(model, 'node')].map(results[model]['diff_strength'])
        df[(model, 'Dd')] = df[(model, 'node')].map(results[model]['diff_degree'])
        df[(model, 'Df')] = df[(model, 'node')].map(results[model]['diff_tf'])
        df[(model, 'Dc')] = df[(model, 'node')].map(results[model]['diff_closeness'])

    for model in results:
        #df[(model, 'Ds')] = (df[(model, 'Ds')]/1000).round(1)
        df[(model, 'Dd')] = (df[(model, 'Dd')]/100).round(1)
        df[(model, 'Df')] = (df[(model, 'Df')]/1000).round(1)
        df[(model, 'Dc')] = df[(model, 'Dc')].round(2)
   
    
    for model in results:
        for metric in ['degree', 'tf', 'closeness']: #add strength here if you need it
            mean = df[(model, metric)].mean()
            std = df[(model, metric)].std()
            if std == 0:
                df[(model, f'{metric}_zscore')] = 0
            else:
                df[(model, f'{metric}_zscore')] = (df[(model, metric)] - mean) / std

    # Step 5: Sort each model's data by nodes
    sorted_dfs = {}
    for model in results:
        model_columns = [(model, col) for col in df.columns.levels[1] if col in ['node','d','f','c', 'degree_zscore', 'tf_zscore','closeness_zscore','Dd','Df','Dc']] #add s and strength_zscore here if you need it
        sorted_dfs[model] = df[model_columns].sort_values(by=(model, 'node')).reset_index(drop=True)
    
    
    ## Step 6: Calculate cosine similarities
    #models = list(results.keys())
    #for i, model1 in enumerate(models):
    #    for j, model2 in enumerate(models):
    #        if i < j:  # Only calculate for unique pairs
    #            vec1 = sorted_dfs[model1][[
    #                (model1, 'strength_zscore'),
    #                #(model1, 'closeness_zscore'),
    #                (model1, 'degree_zscore'),
    #                (model1, 'tf_zscore')
    #            ]].values
    #            vec2 = sorted_dfs[model2][[
    #                (model2, 'strength_zscore'),
    #                #(model2, 'closeness_zscore'),
    #                (model2, 'degree_zscore'),
    #                (model2, 'tf_zscore')
    #            ]].values
    #            
    #            cosine_sim = np.array([
    #                abs(1 - cosine(v1, v2) if not np.isnan(v1).any() and not np.isnan(v2).any() else np.nan)
    #                for v1, v2 in zip(vec1, vec2)
    #            ])
    #            
    #            sorted_dfs[model1][(model1, f'S{model2[0]}')] = cosine_sim
    #            sorted_dfs[model2][(model2, f'S{model1[0]}')] = cosine_sim
        # Step 6: Calculate average cosine similarity
    models = list(results.keys())
    for i, model1 in enumerate(models):
        cosine_sims = []
        for j, model2 in enumerate(models):
            if i != j:  # Calculate for all pairs except self
                vec1 = sorted_dfs[model1][[
                    (model1, 'closeness_zscore'),
                    (model1, 'degree_zscore'),
                    (model1, 'tf_zscore')
                ]].values
                vec2 = sorted_dfs[model2][[
                    (model2, 'closeness_zscore'),
                    (model2, 'degree_zscore'),
                    (model2, 'tf_zscore')
                ]].values
                
                cosine_sim = np.array([
                    abs(1 - cosine(v1, v2) if not np.isnan(v1).any() and not np.isnan(v2).any() else np.nan)
                    for v1, v2 in zip(vec1, vec2)
                ])
                
                cosine_sims.append(cosine_sim)
        
        avg_cosine_sim = np.nanmean(cosine_sims, axis=0)
        sorted_dfs[model1][(model1, 'AvgS')] = avg_cosine_sim          

    #for model in results:
    #    model_columns = [(model, col) for col in ['node', 'strength_zscore', 'closeness_zscore', 'degree_zscore', 'tf_zscore']]
    # Step 7: Sort each model's data by strength_zscore

    for model in results:
        sorted_dfs[model] = sorted_dfs[model].sort_values(by=(model, 'f'), ascending=False).reset_index(drop=True)

    
    # Step 8: Concatenate sorted DataFrames
    df_sorted = pd.concat(sorted_dfs.values(), axis=1)

    for model in results:
        for metric in ['closeness_zscore', 'degree_zscore', 'tf_zscore']:
            del df_sorted[(model, metric)]  


    for column in df_sorted.columns:
        if column[1] != 'AvgS':
            df_sorted[column] = df_sorted[column].astype(str)


    # Split the dataframe into two parts
    models = list(results.keys())
    print(models)
    models = ['gpt-3.5','Haiku','mistral-7b','Llama-3-8B']
    df_first_two = df_sorted.loc[:, models[:2]]
    df_second_two = df_sorted.loc[:, models[2:]]

    captiondictionary={'climate':'Climate Change',
                       'gwarming':'Global warming',
                       'math': 'Math anxiety',
                       'misinformation_health': 'Misinformation in health'}
    captionlanguage={'ita':'Italian',
                     'eng':'English'}

    # Function to create LaTeX table
    def create_latex_table(df, caption):
        caption = f"Top 20 nodes (based on frequency) for the {captiondictionary[folders_of_interest[0]]} topic in {captionlanguage[language]}. "
        caption += f"Columns show node name (node), degree (d, divided by 100), frequency (f, divided by 1000), "
        caption += f"and their Euclidean differences (Dd, Df, Dc) to other models. "
        caption += "AvgS represents the average cosine similarity with other models."
        caption += "Note: d and Dd are shown after division by 100, while f and Df are shown after division by 1000."


        latex_table = "\\begin{table}[htbp]\n\\centering\n\\setlength{\\tabcolsep}{4pt}\n\\footnotesize\n"
        latex_table += df.head(20).to_latex(index=False, float_format="%.2f")
        latex_table += f"\\caption{{{caption}}}\n\\label{{tab:mytable}}\n\\end{{table}}"
        return latex_table

    # Create LaTeX tables
    latex_table1 = create_latex_table(df_first_two, f"Top 20 nodes (based on f) for {folders_of_interest[0]}")
    latex_table2 = create_latex_table(df_second_two, f"Top 20 nodes (based on f) for  {folders_of_interest[0]}")
    #latex_output = df_sorted.head(20).to_latex(index=False, float_format=f"%.1f")
    print(latex_table1, '\n')
    print(latex_table2)

    return df_sorted

In [12]:
def create_standardized_dataframe(results):
    # Step 1-4: Create initial DataFrame and calculate z-scores (unchanged)
    df = pd.concat({k: pd.DataFrame(v['strength'].items(), columns=['node', 'strength']) for k, v in results.items()}, axis=1)

    display(df[df.isna().any(axis=1)])

    # Delete rows with NaN values
    df = df.dropna()

    for model in results:
        df[(model, 'closeness')] = df[(model, 'node')].map(results[model]['closeness'])
        df[(model, 'degree')] = df[(model, 'node')].map(results[model]['degree'])
        df[(model, 'tf')] = df[(model, 'node')].map(results[model]['tf'])
    
    for model in results:
        #df[(model, 's')] = (df[(model, 'strength')]/1000).round(1)
        df[(model, 'd')] = (df[(model, 'degree')]/100).round(1)
        df[(model, 'f')] = (df[(model, 'tf')]/1000).round(1)
        df[(model, 'c')] = df[(model, 'closeness')].round(2)

    for model in results:
        #df[(model, 'Ds')] = df[(model, 'node')].map(results[model]['diff_strength'])
        df[(model, 'Dd')] = df[(model, 'node')].map(results[model]['diff_degree'])
        df[(model, 'Df')] = df[(model, 'node')].map(results[model]['diff_tf'])
        df[(model, 'Dc')] = df[(model, 'node')].map(results[model]['diff_closeness'])

    for model in results:
        #df[(model, 'Ds')] = (df[(model, 'Ds')]/1000).round(1)
        df[(model, 'Dd')] = (df[(model, 'Dd')]/100).round(1)
        df[(model, 'Df')] = (df[(model, 'Df')]/1000).round(1)
        df[(model, 'Dc')] = df[(model, 'Dc')].round(2)
   
    
    for model in results:
        for metric in ['degree', 'tf', 'closeness']: #add strength here if you need it
            mean = df[(model, metric)].mean()
            std = df[(model, metric)].std()
            if std == 0:
                df[(model, f'{metric}_zscore')] = 0
            else:
                df[(model, f'{metric}_zscore')] = (df[(model, metric)] - mean) / std

    # Step 5: Sort each model's data by nodes
    sorted_dfs = {}
    for model in results:
        model_columns = [(model, col) for col in df.columns.levels[1] if col in ['node','d','f','c', 'degree_zscore', 'tf_zscore','closeness_zscore','Dd','Df','Dc']] #add s and strength_zscore here if you need it
        sorted_dfs[model] = df[model_columns].sort_values(by=(model, 'node')).reset_index(drop=True)
    
    
    ## Step 6: Calculate cosine similarities
    #models = list(results.keys())
    #for i, model1 in enumerate(models):
    #    for j, model2 in enumerate(models):
    #        if i < j:  # Only calculate for unique pairs
    #            vec1 = sorted_dfs[model1][[
    #                (model1, 'strength_zscore'),
    #                #(model1, 'closeness_zscore'),
    #                (model1, 'degree_zscore'),
    #                (model1, 'tf_zscore')
    #            ]].values
    #            vec2 = sorted_dfs[model2][[
    #                (model2, 'strength_zscore'),
    #                #(model2, 'closeness_zscore'),
    #                (model2, 'degree_zscore'),
    #                (model2, 'tf_zscore')
    #            ]].values
    #            
    #            cosine_sim = np.array([
    #                abs(1 - cosine(v1, v2) if not np.isnan(v1).any() and not np.isnan(v2).any() else np.nan)
    #                for v1, v2 in zip(vec1, vec2)
    #            ])
    #            
    #            sorted_dfs[model1][(model1, f'S{model2[0]}')] = cosine_sim
    #            sorted_dfs[model2][(model2, f'S{model1[0]}')] = cosine_sim
        # Step 6: Calculate average cosine similarity
    models = list(results.keys())
    for i, model1 in enumerate(models):
        cosine_sims = []
        for j, model2 in enumerate(models):
            if i != j:  # Calculate for all pairs except self
                vec1 = sorted_dfs[model1][[
                    (model1, 'closeness_zscore'),
                    (model1, 'degree_zscore'),
                    (model1, 'tf_zscore')
                ]].values
                vec2 = sorted_dfs[model2][[
                    (model2, 'closeness_zscore'),
                    (model2, 'degree_zscore'),
                    (model2, 'tf_zscore')
                ]].values
                
                cosine_sim = np.array([
                    abs(1 - cosine(v1, v2) if not np.isnan(v1).any() and not np.isnan(v2).any() else np.nan)
                    for v1, v2 in zip(vec1, vec2)
                ])
                
                cosine_sims.append(cosine_sim)
        
        avg_cosine_sim = np.nanmean(cosine_sims, axis=0)
        sorted_dfs[model1][(model1, 'AvgS')] = avg_cosine_sim          

    #for model in results:
    #    model_columns = [(model, col) for col in ['node', 'strength_zscore', 'closeness_zscore', 'degree_zscore', 'tf_zscore']]
    # Step 7: Sort each model's data by strength_zscore

    for model in results:
        sorted_dfs[model] = sorted_dfs[model].sort_values(by=(model, 'f'), ascending=False).reset_index(drop=True)

    
    # Step 8: Concatenate sorted DataFrames
    df_sorted = pd.concat(sorted_dfs.values(), axis=1)

    for model in results:
        for metric in ['closeness_zscore', 'degree_zscore', 'tf_zscore']:
            del df_sorted[(model, metric)]  


    for column in df_sorted.columns:
        if column[1] != 'AvgS':
            df_sorted[column] = df_sorted[column].astype(str)


    # Split the dataframe into two parts
    models = list(results.keys())
    print(models)
    models = ['GPT-3.5', 'GPT-4', 'Haiku', 'Llama-3-8B', 'Llama-3.1-70B', 'Mistral-7b']

    
        # After creating df_sorted and cleaning up columns as you have done above:

    # Get the list of models
    models = list(results.keys())
    print(models)

    # We'll now create three subtables:
    # First subtable: first two models
    # Second subtable: next two models
    # Third subtable: last two models
    df_first_two = df_sorted.loc[:, models[:2]]
    df_second_two = df_sorted.loc[:, models[2:4]]
    df_third_two = df_sorted.loc[:, models[4:]]

    captiondictionary = {
        'climate': 'Climate Change',
        'gwarming': 'Global warming',
        'math': 'Math anxiety',
        'misinformation_health': 'Misinformation in health'
    }
    captionlanguage = {
        'ita': 'Italian',
        'eng': 'English'
    }

    # Function to create LaTeX table
    def create_latex_table(df, caption):
        caption = f"Top 20 nodes (based on frequency) for the {captiondictionary[folders_of_interest[0]]} topic in {captionlanguage[language]}. "
        caption += f"Columns show node name (node), degree (d, divided by 100), frequency (f, divided by 1000), "
        caption += f"and their Euclidean differences (Dd, Df, Dc) to other models. "
        caption += "AvgS represents the average cosine similarity with other models."
        caption += "Note: d and Dd are shown after division by 100, while f and Df are shown after division by 1000."

        latex_table = "\\begin{table}[htbp]\n\\centering\n\\setlength{\\tabcolsep}{4pt}\n\\footnotesize\n"
        latex_table += df.head(20).to_latex(index=False, float_format="%.2f")
        latex_table += f"\\caption{{{caption}}}\n\\label{{tab:mytable}}\n\\end{{table}}"
        return latex_table

    # Create LaTeX tables for each subtable
    latex_table1 = create_latex_table(df_first_two, f"Top 20 nodes (based on f) for {folders_of_interest[0]}")
    latex_table2 = create_latex_table(df_second_two, f"Top 20 nodes (based on f) for {folders_of_interest[0]}")
    latex_table3 = create_latex_table(df_third_two, f"Top 20 nodes (based on f) for {folders_of_interest[0]}")

    print(latex_table1, '\n')
    print(latex_table2, '\n')
    print(latex_table3)

    return df_sorted


In [13]:
pd.set_option('display.max_columns', None)
display(create_standardized_dataframe(euclidean_differences).head(20))

Empty DataFrame
Columns: [(GPT-3.5, node), (GPT-3.5, strength), (GPT-4, node), (GPT-4, strength), (Haiku, node), (Haiku, strength), (Llama-3-8B, node), (Llama-3-8B, strength), (Llama-3.1-70B, node), (Llama-3.1-70B, strength), (Mistral-7b, node), (Mistral-7b, strength)]
Index: []

['GPT-3.5', 'GPT-4', 'Haiku', 'Llama-3-8B', 'Llama-3.1-70B', 'Mistral-7b']
['GPT-3.5', 'GPT-4', 'Haiku', 'Llama-3-8B', 'Llama-3.1-70B', 'Mistral-7b']
\begin{table}[htbp]
\centering
\setlength{\tabcolsep}{4pt}
\footnotesize
\begin{tabular}{lllllllrlllllllr}
\toprule
\multicolumn{8}{r}{GPT-3.5} & \multicolumn{8}{r}{GPT-4} \\
node & d & f & c & Dd & Df & Dc & AvgS & node & d & f & c & Dd & Df & Dc & AvgS \\
\midrule
student & 8.4 & 9.0 & 0.69 & 14.8 & 5.7 & 0.06 & 1.00 & student & 18.5 & 11.3 & 0.68 & 16.0 & 5.0 & 0.06 & 1.00 \\
it & 5.5 & 3.6 & 0.61 & 9.7 & 2.4 & 0.06 & 1.00 & it & 13.2 & 4.9 & 0.62 & 13.7 & 2.6 & 0.09 & 1.00 \\
help & 5.8 & 3.5 & 0.61 & 5.5 & 3.4 & 0.11 & 0.95 & help & 10.9 & 3.2 & 0.59 & 10.4 & 2.8 & 0.07 & 0.98 \\
learning & 3.7 & 2.5 & 0.56 & 5.8 & 0.8 & 0.04 & 0.99 & learning & 8.2 & 3.1 & 0.57 & 7.2 & 1.7 & 0.04 & 0.99 \\
address & 3.8 & 2.5 & 0.56 & 4.7 & 1.3 & 0.05 & 0.99 & mathematic & 7.4 & 2.7 & 0.56 & 5.7 & 4.5 & 0.04 & 1.00 \\
confidence & 3.9 & 2.3 & 0.57 &

GPT-3.5                                                   GPT-4  \
            node    d    f     c    Dd   Df    Dc      AvgS         node   
0        student  8.4  9.0  0.69  14.8  5.7  0.06  0.997780      student   
1             it  5.5  3.6  0.61   9.7  2.4  0.06  0.997844           it   
2           help  5.8  3.5  0.61   5.5  3.4  0.11  0.950538         help   
3       learning  3.7  2.5  0.56   5.8  0.8  0.04  0.987605     learning   
4        address  3.8  2.5  0.56   4.7  1.3  0.05  0.988725   mathematic   
5     confidence  3.9  2.3  0.57   3.4  2.4  0.08  0.941771      address   
6       positive  2.4  2.1  0.53   2.0  2.3  0.05  0.930508         they   
7       educator  3.5  2.1  0.56   5.2  1.8  0.06  0.969953  environment   
8         create  5.3  2.1   0.6   8.6  2.5  0.04  0.998476  educational   
9      education  2.6  2.1  0.54   7.1  2.3  0.02  0.989882      ability   
10       ability  3.9  1.9  0.57   5.2  1.0  0.06  0.986810   experience   
11       provide  4.6  1.8  0.59   6.1  0.9  0.05  0.983758     attitude   
12          fear  3.9  1.8  0.57   5.8  1.2  0.06  0.987007       parent   
13   environment  3.0  1.8  0.55   8.0  1.1  0.03  0.978823         fear   
14       support  4.0  1.7  0.57   4.1  0.8  0.07  0.988687      teacher   
15         issue  3.2  1.7  0.55   4.8  2.1  0.05  0.984562     educator   
16    experience  5.0  1.6  0.59   6.1  1.3  0.07  0.988920  performance   
17    mathematic  2.7  1.6  0.54  10.0  6.4  0.06  0.992099    education   
18    supportive  1.6  1.5  0.51   2.3  0.7  0.04  0.989161       create   
19  mathematical  2.8  1.5  0.54   5.6  2.9  0.03  0.992598     teaching   

                                                        Haiku                  \
       d     f     c    Dd   Df    Dc      AvgS          node    d    f     c   
0   18.5  11.3  0.68  16.0  5.0  0.06  0.998426       student  9.2  7.8  0.67   
1   13.2   4.9  0.62  13.7  2.6  0.09  0.997974    mathematic  5.0  3.7  0.58   
2   10.9   3.2  0.59  10.4  2.8  0.07  0.976004       address  4.1  3.4  0.56   
3    8.2   3.1  0.57   7.2  1.7  0.04  0.990483  mathematical  4.2  3.2  0.56   
4    7.4   2.7  0.56   5.7  4.5  0.04  0.995061            it  5.0  3.0  0.58   
5    7.6   2.7  0.56   6.5  1.2  0.04  0.989653      approach  5.2  2.9  0.58   
6   10.6   2.6  0.59   9.8  1.9  0.04  0.989921       problem  4.6  2.8  0.57   
7    8.7   2.5  0.57   8.6  1.4  0.05  0.993673      societal  2.8  2.5  0.54   
8    7.8   2.5  0.56  11.5  4.2  0.11  0.875803        factor  2.8  2.3  0.53   
9    8.7   2.5  0.57   9.0  2.0  0.07  0.988615     education  2.6  2.2  0.53   
10  10.2   2.5  0.59   9.8  2.3  0.06  0.978470       develop  5.4  2.1  0.59   
11   7.6   2.4  0.56   8.9  3.2  0.08  0.924383         issue  2.6  2.1  0.53   
12   5.8   2.3  0.54   7.7  3.7  0.08  0.838155       subject  4.9  2.1  0.58   
13   8.9   2.3  0.57   9.7  1.6  0.07  0.991593        create  5.8  2.0  0.59   
14   7.4   2.3  0.56   6.6  2.5  0.03  0.945775      learning  4.6  2.0  0.57   
15   7.1   2.2  0.55   6.7  1.9  0.04  0.978088    individual  3.9  1.9  0.56   
16   6.6   2.1  0.55   8.5  2.7  0.08  0.962257          they  5.1  1.8  0.58   
17   6.1   2.1  0.55   5.8  2.4  0.03  0.985753      attitude  3.4  1.7  0.55   
18   9.8   2.1  0.58   7.0  2.4  0.03  0.995171       ability  4.4  1.7  0.57   
19   6.3   2.0  0.55   7.4  2.4  0.06  0.936777     challenge  5.1  1.6  0.58   

                                Llama-3-8B                                    \
      Dd   Df    Dc      AvgS         node     d     f     c   Dd   Df    Dc   
0   13.3  8.0  0.04  0.998861      student  14.5  12.5  0.64  9.8  6.8  0.07   
1    5.8  3.7  0.06  0.994376           it   8.3   3.7  0.56  6.9  2.2  0.08   
2    4.1  2.1  0.05  0.969939           we   5.9   3.5  0.53  7.2  5.8  0.05   
3    3.2  2.5  0.06  0.982176       create   9.2   3.4  0.57  6.2  2.8  0.04   
4   10.7  3.5  0.05  0.998085   mathematic   6.0   3.1  0.54  5.0  4.0  0